In [ ]:
!pip install transformers
!pip install kaggle


In [ ]:
# Download the Kaggle dataset (replace with your dataset's name and path)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'mexwell/food-vitamins-minerals-macronutrient'

# Unzip the dataset (adjust the file name if needed)
!unzip food-vitamins-minerals-macronutrient.zip
#!ls

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/mexwell/food-vitamins-minerals-macronutrient
License(s): GPL-2.0
food-vitamins-minerals-macronutrient.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  food-vitamins-minerals-macronutrient.zip
  inflating: food.csv                


In [ ]:
# Import necessary libraries
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import pickle

# Load the dataset into a pandas DataFrame
df = pd.read_csv('food.csv')

# Specify columns to remove
columns_to_remove = ['Nutrient Data Bank Number', 'Data.Alpha Carotene', 'Data.Beta Carotene', 'Data.Beta Cryptoxanthin', 'Data.Choline', 'Data.Lutein and Zeaxanthin', 'Data.Lycopene', 'Data.Niacin', 'Data.Retinol', 'Data.Riboflavin', 'Data.Selenium', 'Data.Thiamin', 'Data.Water', 'Data.Major Minerals.Calcium', 'Data.Major Minerals.Copper', 'Data.Major Minerals.Iron', 'Data.Major Minerals.Magnesium', 'Data.Major Minerals.Phosphorus', 'Data.Major Minerals.Potassium', 'Data.Major Minerals.Sodium', 'Data.Major Minerals.Zinc', 'Data.Vitamins.Vitamin A - RAE', 'Data.Vitamins.Vitamin B12', 'Data.Vitamins.Vitamin B6', 'Data.Vitamins.Vitamin C', 'Data.Vitamins.Vitamin E', 'Data.Vitamins.Vitamin K']

# Remove specified columns
df = df.drop(columns=columns_to_remove)

# Get text data from the second column (food ingredient names)
food_ingredients = df.iloc[:, 1].tolist()

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=50)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()  # Get [CLS] token embeddings
    return embeddings

# Get BERT embeddings for food ingredients
ingredient_embeddings = [get_bert_embeddings(ingredient) for ingredient in food_ingredients]

# Create a new DataFrame with food ingredients and their embeddings
embeddings_df = pd.DataFrame({'food_ingredient': food_ingredients, 'embedding': ingredient_embeddings})



KeyboardInterrupt: 

In [ ]:
# Add embeddings to the preprocessed DataFrame
df['embedding'] = ingredient_embeddings

# Save the updated DataFrame to a pickle file
with open('preprocessed_data_with_embeddings.pkl', 'wb') as file:
    pickle.dump(df, file)

In [ ]:
print("Preprocessed CSV:")
display(df.head())

Preprocessed CSV:


,Category,Description,Data.Carbohydrate,Data.Cholesterol,Data.Fiber,Data.Protein,Data.Sugar Total,Data.Fat.Monosaturated Fat,Data.Fat.Polysaturated Fat,Data.Fat.Saturated Fat,Data.Fat.Total Lipid,embedding
0,Milk,"Milk, human",6.89,14,0.0,1.03,6.89,1.658,0.497,2.009,4.38,"[[-0.4338273, -0.102978416, -0.38401067, 0.202..."
1,Milk,"Milk, NFS",4.87,8,0.0,3.34,4.89,0.426,0.065,1.164,1.99,"[[-0.51686615, -0.1436615, -0.31570256, -0.029..."
2,Milk,"Milk, whole",4.67,12,0.0,3.28,4.81,0.688,0.108,1.860,3.20,"[[-0.4939267, 0.08810828, -0.18735683, -0.0058..."
3,Milk,"Milk, low sodium, whole",4.46,14,0.0,3.10,4.46,0.999,0.128,2.154,3.46,"[[-0.57828254, -0.025768813, -0.12763183, -0.0..."
4,Milk,"Milk, calcium fortified, whole",4.67,12,0.0,3.28,4.81,0.688,0.108,1.860,3.20,"[[-0.5756594, -0.24423215, -0.12151379, -0.128..."
